In [1]:
import pandas as pd
import sklearn
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import xgboost as xgb

e:\python352\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# constant definition
# small_data_path = './data/small_size/sample_atec_anti_fraud_train.csv'
full_data_path = '../data/full_size/atec_anti_fraud_train.csv'

In [3]:
# write sample data to file
df = pd.read_csv(full_data_path,index_col = 0)
# sdf.to_csv(small_data_path)

In [4]:
# preprocess sample data
# method 1: delete all -1 labeled data
psdf1 = df[df['label']!=-1]
# fill in all NAN with most common value
psdf1 = psdf1.fillna(psdf1.mode().iloc[0])

In [5]:
X = psdf1.iloc[:,2:]
Y = psdf1.iloc[:,0]
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=1)

In [6]:
# preprocessing scale to 1
scaler = preprocessing.StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train),index = x_train.index,columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test),index = x_test.index,columns = x_test.columns)

In [7]:
from xgboost import XGBClassifier
clf = XGBClassifier(loss = 'log',verbose=True,class_weight = 'balanced')
print('fitting')
clf.fit(x_train,y_train)
from sklearn import metrics
print('predicting')
y_train_predict = clf.predict(x_train)
print(metrics.accuracy_score(y_train,y_train_predict))

fitting


XGBoostError: b'[02:18:15] d:\\build\\xgboost\\xgboost-0.71.git\\src\\objective\\objective.cc:23: Unknown objective function binary'

In [7]:
y_test_predict = clf.predict_proba(x_test)
result = pd.DataFrame({'score':y_test_predict[:,1],'truth':y_test},index = x_test.index)
result.to_csv('./result/large_binary_cw_xgb_no_time_result.csv')

NameError: name 'clf' is not defined

In [9]:
# save sgd model
from sklearn.externals import joblib
model_name = 'sgdclassifier.pkl'
joblib.dump(clf,model_name,compress=9)

['sgdclassifier.pkl']

In [2]:
# import model
clf = joblib.load(model_name)

In [53]:
yp_pedict = clf.predict_proba(x_test)

e:\python352\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [54]:
# import real test data
test_data = pd.read_csv('./data/full_size/atec_anti_fraud_test_a.csv',index_col = 0)

In [55]:
# fill nan
test_data = test_data.fillna(test_data.mode().iloc[0])
test_data_x = test_data.iloc[:,1:]
test_predict_y = clf.predict_proba(test_data_x)

e:\python352\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [16]:
from sklearn import metrics
test_y = clf.predict_proba(test_data_x)

In [56]:
result = pd.DataFrame({'score':test_predict_y[:,1]},index=test_data.index)

In [58]:
result.to_csv('bm.csv')

In [65]:
result['score'].mean()

6.1016783683298489e-06

In [60]:
result['score']

id
8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6fda5578f0e340908fa4    0.0
f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7c335237327bb02e02b    0.0
ee85f808b1fd49eaba308527e1686c509dc8e3e50574888e0f5f8bc21733711c    0.0
0df905aa187938d56a9b0816b13f54ac1f87d658a33cd06505b738c66b54fd3f    0.0
15f532f979c4f092bbbe28e5409c8c3b8454ece8f1ab3ee00755b4949ef75c03    0.0
614728f2aba299d20ddffca7e1ebc38ae8f239663562a9a3602e1e2c373f7299    0.0
3dc22d969895bc4afdaa5266d1586ae1a360adfcc638b5fb1e2f242bbbc869eb    0.0
b6cba0173d6a894f443578b5d82c00444dfe24c5e1705d4b2b835062eeb61d7a    0.0
84d010a48c747947588589473e2ebdc59db3d348849c5b67759dfbcd1d92f35e    0.0
bc00fec7ff8b3e4f8bd826048da109e7fe40fcc604ba28ce78576e05a4e67fba    0.0
85426887c8a4f4fd881ae1e79654940e35c68bfd681a5eaac4787f0f82483780    0.0
ee84ac886d80e779aeb5c4af65dbb8e087014d7c98b7973bd9d91d8e2467a341    0.0
4cbb83002d62e3257d49d3742b0a27eb677ae1c464f69e4b19fe73d095ab8eae    0.0
4e6ba7f2c84040ec8afde9f42c5bee3d53fd24e3df292980a6475fc06138d